In [94]:
# Importing necessary libraries

import mlflow
from mlflow.tracking import MlflowClient

from keras.models import load_model, save_model
from tensorflow import keras
import os
import numpy as np
import matplotlib.pyplot as plt
import random

from fastapi import FastAPI, UploadFile, File
from PIL import Image
import io

In [10]:
client = MlflowClient(tracking_uri='http://localhost:8080') 
experiments = client.search_experiments()


In [16]:
for experiment in experiments:
    print("Experiment ID:", experiment.experiment_id)
    print("Name:", experiment.name)
    print("Artifact Location:", experiment.artifact_location)
    print("Lifecycle Stage:", experiment.lifecycle_stage)
    print("-------------------------------")

Experiment ID: 185101760259181866
Name: MNIST models manual
Artifact Location: mlflow-artifacts:/185101760259181866
Lifecycle Stage: active
-------------------------------
Experiment ID: 439877701008752385
Name: MNIST models auto
Artifact Location: mlflow-artifacts:/439877701008752385
Lifecycle Stage: active
-------------------------------
Experiment ID: 0
Name: Default
Artifact Location: mlflow-artifacts:/0
Lifecycle Stage: active
-------------------------------


In [27]:
experiment_id = '439877701008752385'

runs = mlflow.search_runs(experiment_id)


In [34]:
for run in runs.iterrows():
    print("Run ID:", run[1]['run_id'])
    print("Status:", run[1]['status'])
    print("Start Time:", run[1]['start_time'])
    print("End Time:", run[1]['end_time'])
    print("Validation Loss:",run[1]['metrics.val_loss'])
    print("-------------------------------------")

Run ID: c1dc5de025ce4b5a9c0a79eadefd5be7
Status: FINISHED
Start Time: 2024-04-19 20:21:58.381000+00:00
End Time: 2024-04-19 20:25:22.824000+00:00
Validation Loss: 2.406048059463501
-------------------------------------
Run ID: b466a1cadf134e799cb2fae68bf3f62c
Status: FINISHED
Start Time: 2024-04-19 20:21:38.878000+00:00
End Time: 2024-04-19 20:21:58.057000+00:00
Validation Loss: 1.0681698322296143
-------------------------------------
Run ID: b404f21f96e546b6b7cba5ad698fe5e3
Status: FINISHED
Start Time: 2024-04-19 20:18:10.495000+00:00
End Time: 2024-04-19 20:21:35.951000+00:00
Validation Loss: 0.26858389377593994
-------------------------------------
Run ID: 8eb57d9e051c4f7b8234b07d2563b26f
Status: FINISHED
Start Time: 2024-04-19 20:14:46.427000+00:00
End Time: 2024-04-19 20:18:07.411000+00:00
Validation Loss: 0.3429158329963684
-------------------------------------
Run ID: 72ef047c9d644ac7985090847bb02b73
Status: FINISHED
Start Time: 2024-04-19 20:11:25.532000+00:00
End Time: 2024-04

In [36]:
# Picking the model with the least validation loss

best_loss = float('inf')
best_run_id = None

for run in runs.iterrows():
    loss = run[1]['metrics.val_loss']

    if loss < best_loss:
        best_loss = loss
        best_run_id = run[1]['run_id']

print("Best Run ID:", best_run_id)
print("Best Validation Loss:", best_loss)

Best Run ID: 5ce1fdb66d434e93a13873529b5cfd5c
Best Validation Loss: 0.06834544986486435


In [38]:
model = mlflow.keras.load_model("runs:/" + best_run_id + "/model")

2024/04/20 20:39:13 INFO mlflow.store.artifact.artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false
2024-04-20 20:39:13.582246: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2024-04-20 20:39:13.582283: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-04-20 20:39:13.582295: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-04-20 20:39:13.582591: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-20 20:39:13.582624: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [45]:
# Defining a function to save a keras model on a local machine

def save_model_to_disk(model,file_path,model_name):
    os.makedirs(file_path,exist_ok=True)
    save_path = os.path.join(file_path,model_name)

    save_model(model,save_path)

save_model_to_disk(model,'Models','MNIST_model.keras')
